In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [5]:
def create_sequences(df, feature_cols, target_col, window):
    X, y = [], []

    arr = df[feature_cols].values      # convert ONCE
    target_arr = df[target_col].values # convert ONCE

    for i in range(len(df) - window):
        X.append(arr[i:i+window])
        y.append(target_arr[i+window])

    return np.array(X), np.array(y)


In [6]:
import pandas as pd
df = pd.read_csv("E://PHD/Course materials/Sem 1/Data Mining/Assignment/workspace/stock_price_trend_prediction/stock_price_pred_data_mining/Data/05_cluster_output/cluster_output_Agglomerative.csv")
# 1. Compute next-week (5 trading days) forward return per stock
df['next_week_close'] = df.groupby('stock_id')['Close'].shift(-5)
df['next_week_return'] = (df['next_week_close'] - df['Close']) / df['Close']
# 2. Binary target: 1 = uptrend, 0 = downtrend or flat
df["target"] = (df["Close"].shift(-5) > df["Close"]).astype(int)
# 3. Drop rows where we can't compute future return (last 5 days per stock)
df = df.dropna(subset=['next_week_return'])
# Optional: drop helper column
df = df.drop(columns=['next_week_close'])
drop_cols = ['Date', 'stock_id', 'agg_cluster', 'target', 'next_week_return']
feature_cols = [c for c in df.columns if c not in drop_cols]

print("Number of features:", len(feature_cols))
print("Features:", feature_cols)

Number of features: 23
Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'MA21', 'EMA20', 'EMA50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower', 'CCI_14', 'CMF_20', 'Stoch_K', 'Stoch_D', 'Momentum_10', 'Daily_Return', 'Log_Return']


In [7]:
df.shape

(72548, 28)

In [8]:
from sklearn.metrics import roc_auc_score

WINDOW = 20  # recommended
cluster_lstm_models = {}

for cl in df['agg_cluster'].unique():

    print(f"\n🔵 Training LSTM for Cluster {cl}")
    cluster_df = df[df['agg_cluster'] == cl].copy().sort_values(["stock_id", "Date"])
    
    X_all, y_all = [], []

    # ---- SEQUENCE BUILDING PER STOCK (memory-safe)
    for sid, temp in cluster_df.groupby("stock_id"):
        if len(temp) <= WINDOW + 2:
            continue
        X_temp, y_temp = create_sequences(temp, feature_cols, 'target', WINDOW)
        X_all.append(X_temp)
        y_all.append(y_temp)

    if not X_all:
        print(f"No usable data for cluster {cl}")
        continue

    # Combine results
    X = np.vstack(X_all)
    y = np.concatenate(y_all)

    print(f"Cluster {cl}: Final sequence shape = {X.shape}")

    # ---- Train-test split (time-based)
    split_idx = int(0.7 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # ---- Build LSTM Model
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=(WINDOW, len(feature_cols))),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # ---- Fit model
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=30,
        batch_size=128,
        callbacks=[es],
        verbose=1
    )

    # ---- Store model
    cluster_lstm_models[cl] = model

    # ---- Evaluate
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec  = recall_score(y_test, y_pred, zero_division=0)
    f1   = f1_score(y_test, y_pred, zero_division=0)

    try:
        rocauc = roc_auc_score(y_test, y_pred_prob)
    except:
        rocauc = 0.0   # when only 1 class in test set

    print(f"ROC-AUC: {rocauc:.4f}")

    print(f"""
    🔹 Cluster {cl} Evaluation:
       Accuracy:  {acc:.4f}
       Precision: {prec:.4f}
       Recall:    {rec:.4f}
       Macro-F1:  {f1:.4f}
    """)



🔵 Training LSTM for Cluster 3
Cluster 3: Final sequence shape = (40538, 20, 23)
Epoch 1/30


c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5101 - loss: 0.6974 - val_accuracy: 0.4974 - val_loss: 0.6975
Epoch 2/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5208 - loss: 0.6930 - val_accuracy: 0.4954 - val_loss: 0.6955
Epoch 3/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5203 - loss: 0.6924 - val_accuracy: 0.5025 - val_loss: 0.6944
Epoch 4/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5259 - loss: 0.6919 - val_accuracy: 0.4989 - val_loss: 0.6944
Epoch 5/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5264 - loss: 0.6918 - val_accuracy: 0.4991 - val_loss: 0.6946
Epoch 6/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5276 - loss: 0.6916 - val_accuracy: 0.5044 - val_loss: 0.6936
Epoch 7/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5254 - loss: 0.6918 - val_accuracy: 0.5009 - val_loss: 0.6934
Epoch 8/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5267 - loss: 0.6914 - val_accuracy: 0.4972 - val

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4899 - loss: 0.7209 - val_accuracy: 0.5140 - val_loss: 0.7026
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5544 - loss: 0.6937 - val_accuracy: 0.5140 - val_loss: 0.6969
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5381 - loss: 0.6917 - val_accuracy: 0.5140 - val_loss: 0.6953
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5503 - loss: 0.6873 - val_accuracy: 0.5140 - val_loss: 0.6973
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5605 - loss: 0.6913 - val_accuracy: 0.5140 - val_loss: 0.6962
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5512 - loss: 0.6890 - val_accuracy: 0.5140 - val_loss: 0.6977
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5442 - loss: 0.6911 - val_accuracy: 0.5140 - val_loss: 0.6956
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5506 - loss: 0.6894 - val_accuracy: 0.5140 - val_loss: 0.

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5074 - loss: 0.7014 - val_accuracy: 0.5561 - val_loss: 0.6885
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5006 - loss: 0.7050 - val_accuracy: 0.5561 - val_loss: 0.6917
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4977 - loss: 0.6997 - val_accuracy: 0.5561 - val_loss: 0.6915
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5036 - loss: 0.6996 - val_accuracy: 0.5561 - val_loss: 0.6910
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5232 - loss: 0.6988 - val_accuracy: 0.5561 - val_loss: 0.6886
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5117 - loss: 0.6980 - val_accuracy: 0.5561 - val_loss: 0.6915
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC-AUC: 0.4988

    🔹 Cluster 2 Evaluation:
       Accuracy:  0.5659
       Precision: 0.5689
       Recall:    0.9833
       Macro-F1:  0.7208
    

🔵 Training LSTM for Cluster 1
Cluster 1: Final sequence sha

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5112 - loss: 0.7066 - val_accuracy: 0.5123 - val_loss: 0.6929
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4982 - loss: 0.7000 - val_accuracy: 0.5123 - val_loss: 0.6929
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5080 - loss: 0.6975 - val_accuracy: 0.5118 - val_loss: 0.6929
Epoch 4/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5060 - loss: 0.6955 - val_accuracy: 0.4877 - val_loss: 0.6936
Epoch 5/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5089 - loss: 0.6943 - val_accuracy: 0.5118 - val_loss: 0.6929
Epoch 6/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5048 - loss: 0.6938 - val_accuracy: 0.5118 - val_loss: 0.6929
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5072 - loss: 0.6937 - val_accuracy: 0.5118 - val_loss: 0.6932
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5107 - loss: 0.6938 - val_accuracy: 0.5118 - val_loss: 0.6930
1

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4957 - loss: 0.7285 - val_accuracy: 0.5510 - val_loss: 0.6880
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5123 - loss: 0.7065 - val_accuracy: 0.5510 - val_loss: 0.6884
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5158 - loss: 0.7028 - val_accuracy: 0.5510 - val_loss: 0.6883
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5158 - loss: 0.6993 - val_accuracy: 0.5510 - val_loss: 0.6887
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5110 - loss: 0.6953 - val_accuracy: 0.5510 - val_loss: 0.6880
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5158 - loss: 0.6964 - val_accuracy: 0.5510 - val_loss: 0.6884
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC-AUC: 0.5046

    🔹 Cluster 0 Evaluation:
       Accuracy:  0.4515
       Precision: 0.4517
       Recall:    0.9987
       Macro-F1:  0.6221
    
